# Złożona kwadratura Simpsona

In [23]:
import numpy as np
from scipy.integrate import quad, trapezoid

def kwadratura_simpsona(x, y):
    if len(x) < 2:
        raise ValueError("Potrzebne są co najmniej 2 punkty.")

    n = len(x) - 1
    integral = 0.0

    for i in range(n):
        a, b = x[i], x[i+1]
        fa, fb = y[i], y[i+1]

        mid_point = (a + b) / 2

        if mid_point in x:
            mid_idx = np.where(np.isclose(x, mid_point))[0][0]
            f_mid = y[mid_idx]
        else:
            # Interpolacja liniowa dla punktu środkowego
            idx = np.searchsorted(x, mid_point)
            if idx == 0:
                f_mid = y[0]
            elif idx == len(x):
                f_mid = y[-1]
            else:
                x1, x2 = x[idx-1], x[idx]
                y1, y2 = y[idx-1], y[idx]
                f_mid = y1 + (mid_point - x1) * (y2 - y1) / (x2 - x1)

        integral += (b - a) / 6 * (fa + 4 * f_mid + fb)

    return integral

In [24]:

def f1(x):
    return np.exp(-x**2) * (np.log(x))**2

def f2(x):
    return 1 / (x**3 - 2*x - 5)

def f3(x):
    return x**5 * np.exp(-x) * np.sin(x)


# --- Parametry całkowania ---
# Liczba węzłów (musi być nieparzysta)
N = 101

test_cases = [
    {
        "func": f1,
        "interval": (0.1, 5),
        "name": "f1(x) = exp(-x^2) * (log(x))^2",
    },
    {
        "func": f2,
        "interval": (-1.5, -0.5),
        "name": "f2(x) = 1 / (x^3 - 2x - 5)",
    },
    {
        "func": f3,
        "interval": (0, 10),
        "name": "f3(x) = x^5 * exp(-x) * sin(x)",
    }
]

# --- Pętla porównawcza ---
for case in test_cases:
    a, b = case["interval"]
    func = case["func"]

    # Generowanie równoodległych węzłów i wartości
    x = np.linspace(a, b, N)
    y = func(x)

    # Obliczenie całek różnymi metodami
    val_simpson = kwadratura_simpsona(x, y)
    val_trapez = trapezoid(y, x)
    val_quad, err_quad = quad(func, a, b) # quad zwraca wartość i szacowany błąd


    exact = val_quad

    # Obliczenie błędów bezwzględnych
    error_simpson = abs(val_simpson - exact)
    error_trapez = abs(val_trapez - exact)
    error_quad = abs(val_quad - exact)

    # Prezentacja wyników
    print(f"--- Porównanie dla funkcji: {case['name']} na przedziale [{a}, {b}] ---")
    print(f"Liczba węzłów: {N}")
    print(f"Dokładna wartość:               {exact:.10f}")
    print("-" * 60)
    print(f"Metoda Trapezów (trapezoid):    {val_trapez:.10f} | Błąd: {error_trapez:.2e}")
    print(f"Kwadratura Simpsona (własna):   {val_simpson:.10f} | Błąd: {error_simpson:.2e}")
    print(f"Kwadratura Adaptacyjna (quad):  {val_quad:.10f} | Błąd: {error_quad:.2e}")
    print("\n")

--- Porównanie dla funkcji: f1(x) = exp(-x^2) * (log(x))^2 na przedziale [0.1, 5] ---
Liczba węzłów: 101
Dokładna wartość:               0.7591621210
------------------------------------------------------------
Metoda Trapezów (trapezoid):    0.7683813510 | Błąd: 9.22e-03
Kwadratura Simpsona (własna):   0.7683813510 | Błąd: 9.22e-03
Kwadratura Adaptacyjna (quad):  0.7591621210 | Błąd: 0.00e+00


--- Porównanie dla funkcji: f2(x) = 1 / (x^3 - 2x - 5) na przedziale [-1.5, -0.5] ---
Liczba węzłów: 101
Dokładna wartość:               -0.2371903170
------------------------------------------------------------
Metoda Trapezów (trapezoid):    -0.2371883346 | Błąd: 1.98e-06
Kwadratura Simpsona (własna):   -0.2371883346 | Błąd: 1.98e-06
Kwadratura Adaptacyjna (quad):  -0.2371903170 | Błąd: 0.00e+00


--- Porównanie dla funkcji: f3(x) = x^5 * exp(-x) * sin(x) na przedziale [0, 10] ---
Liczba węzłów: 101
Dokładna wartość:               -10.8881015445
-----------------------------------------------